In [111]:
# Importar bibliotecas

import pandas as pd
from pulp import *

In [110]:
# Crear/leer datos

def leer_datos():
    # Aquí lógica para leer los datos desde un archivo o base de datos
    proyectos = pd.DataFrame({       
        'eslora': [120, 100],
        'manga': [18, 15]}, 
        index=['PRO1', 'PRO2'])

    periodos = pd.DataFrame({
        'tipo_desc': ['FLOTE', 'FLOTE'],
        'fecha_inicio': ['2025-08-08', '2025-08-10'],
        'fecha_fin': ['2025-08-20', '2025-08-16'],
        'proyecto_id': ['PRO1', 'PRO2'],
        'periodo_id': [0, 0]})

    periodos.index = periodos['proyecto_id'] + '_' + periodos['periodo_id'].astype(str)

    muelles = pd.DataFrame({
        'longitud': [130, 110],
        'ancho': [20, 20]},
        index=['SUR', 'NORTE'])

    fecha_inicial = periodos['fecha_inicio'].min()

    return proyectos, periodos, muelles, fecha_inicial


In [120]:
# Preprocesar datos

def preprocesar_datos(proyectos, periodos, muelles, fecha_inicial):

    # Convertir fechas a integer
    periodos['fecha_inicio'] = periodos['fecha_inicio'].apply(lambda x: (pd.Timestamp(x)-pd.Timestamp(fecha_inicial)).days)
    periodos['fecha_fin'] = periodos['fecha_fin'].apply(lambda x: (pd.Timestamp(x)-pd.Timestamp(fecha_inicial)).days)

    dias = list(range(periodos['fecha_inicio'].min(), periodos['fecha_fin'].max()+1))

    return proyectos, periodos, muelles, dias

In [101]:
# Definir variables de decisión

def definir_variables(dias, periodos, muelles):
    x_dpm = LpVariable.dicts("x", (dias, periodos.index, muelles.index), cat='Binary')
    return x_dpm

In [102]:
# Definir la función objetivo

def definir_funcion_objetivo(x_dpm, dias, periodos, muelles):
    objetivo = lpSum(x_dpm[d][p][m] for d in dias for p in periodos.index for m in muelles.index)
    return objetivo

In [103]:
# Definir restricciones

def definir_restricciones(x_dpm, dias, periodos, muelles, proyectos):

    restricciones = {}

    # Cada proyecto como mucho en un muelle por día
    restricciones['1_por_día'] = []
    for d in dias:
        for p in periodos.index:
            restricciones['1_por_día'].append(lpSum(x_dpm[d][p][m] for m in muelles.index) <= 1)
    
    # Manga del proyecto no puede exceder las dimensiones del muelle
    restricciones['Manga_Muelle'] = []
    for d in dias:
        for p in periodos.index:
            for m in muelles.index:
                restricciones['Manga_Muelle'].append(x_dpm[d][p][m] * proyectos.loc[periodos.loc[p,'proyecto_id'], 'manga'] <= muelles.loc[m, 'ancho'])
    
    # Cada proyecto solo puede estar asignado entre su inicio y fin
    restricciones['Periodo_Asignacion'] = []
    for d in dias:
        for p in periodos.index:
            for m in muelles.index:
                if d < periodos.loc[p, 'fecha_inicio'] or d > periodos.loc[p, 'fecha_fin']:
                    restricciones['Periodo_Asignacion'].append(x_dpm[d][p][m] == 0)

    # Los barcos en el mismo muelle no pueden exceder la longitud del muelle
    restricciones['Longitud_Muelle'] = []
    for d in dias:
        for m in muelles.index:
            restricciones['Longitud_Muelle'].append(lpSum(x_dpm[d][p][m] * proyectos.loc[periodos.loc[p,'proyecto_id'], 'eslora'] for p in periodos.index) <= muelles.loc[m, 'longitud'])

    return restricciones

In [ ]:
# Resolver el problema de optimización

def resolver_problema(objetivo, restricciones):

    prob = LpProblem("Asignación de Periodos a Muelles", LpMaximize)
    prob += objetivo

    for constraint in restricciones.values():
        for c in constraint:
            prob += c

    prob.solve()

    return prob

In [105]:
# Imprimir asignación de proyectos por muelle

def imprimir_asignacion(prob, x_dpm, dias, periodos, muelles):
    
    print("Estado de la solución:", LpStatus[prob.status])
    print("\nAsignación de Proyectos a Muelles:\n")
    print("Día\t", "\t".join(muelles.index))

    for d in dias:
        row = f"{d}\t"
        for p in periodos.index:
            for m in muelles.index:
                if x_dpm[d][p][m].varValue == 1:
                    row += f"{p}\t"
                    break
        print(row)


In [108]:
# Optimize

def optimize():
    proyectos, periodos, muelles, fecha_inicial = leer_datos()
    proyectos, periodos, muelles, dias = preprocesar_datos(proyectos, periodos, muelles, fecha_inicial)
    
    x_dpm = definir_variables(dias, periodos, muelles)
    objetivo = definir_funcion_objetivo(x_dpm, dias, periodos, muelles)
    restricciones = definir_restricciones(x_dpm, dias, periodos, muelles, proyectos)
    
    prob = resolver_problema(objetivo, restricciones)
    imprimir_asignacion(prob, x_dpm, dias, periodos, muelles)

In [ ]:
optimize()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/miniconda3/envs/NAME_ENV/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/x4/4ffrr56n3jzb55xn5fzcplwc0000gn/T/4b7a7fb5d23a4172a21f2c4494bb7ca6-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/x4/4ffrr56n3jzb55xn5fzcplwc0000gn/T/4b7a7fb5d23a4172a21f2c4494bb7ca6-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 121 COLUMNS
At line 446 RHS
At line 563 BOUNDS
At line 616 ENDATA
Problem MODEL has 116 rows, 52 columns and 168 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 20 - 0.00 seconds
Cgl0002I 12 variables fixed
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -20 to -1.79769e+308
Probing was

/opt/miniconda3/envs/NAME_ENV/lib/python3.10/site-packages/pulp/pulp.py:1455: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
